In [ ]:
!pip install openfisca-us

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 679 kB 14.6 MB/s 
     |████████████████████████████████| 206 kB 41.0 MB/s 
     |████████████████████████████████| 248 kB 61.9 MB/s 
     |████████████████████████████████| 15.3 MB 57.2 MB/s 
     |████████████████████████████████| 79 kB 8.3 MB/s 
     |████████████████████████████████| 45 kB 3.7 MB/s 
  Created wheel for pytest-dependency: filename=pytest_dependency-0.5.1-py3-none-any.whl size=8218 sha256=33ac793cf9af2a8bd39090e10eadcdb7af66ea8317ed605421a13289762c113e
  Stored in directory: /root/.cache/pip/wheels/7e/72/eb/c96a0b4b22f42d092914ba8fe7b4c639443ef02b529dbbefcf
Successfully built pytest-dependency
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 4.2.0
    Uninstalling typing-extensions-4.2.0:
      Successfully uninstalled typing-extensions-4.2.0
  Attempting uninstall: numpy
    Found exist

In [ ]:
from openfisca_us import Microsimulation

sim = Microsimulation()

sim.calc("spm_unit_net_income")

/usr/local/lib/python3.7/dist-packages/openfisca_core/parameters/config.py:17: LibYAMLWarning: libyaml is not installed in your environment. This can make OpenFisca slower to start. Once you have installed libyaml, run 'pip uninstall pyyaml && pip install pyyaml --no-cache-dir' so that it is used in your Python environment.

  warnings.warn(" ".join(message), LibYAMLWarning)
INFO:root:Generating raw CPS for year 2020.
Downloaded ASEC: 100%|██████████| 169M/169M [00:03<00:00, 54.7MiB/s]


          value       weight
0       28327.0   687.710022
1       76811.0  1516.949951
2       15350.0  1483.560059
3       54856.0   928.729980
4       27050.0  1467.910034
...         ...          ...
65615  152253.0   470.359985
65616   44606.0   312.350006
65617   17282.0   514.109985
65618   78821.0   516.250000
65619   53152.0   386.369995

[65620 rows x 2 columns]

In [ ]:
from openfisca_us.api import *
from openfisca_us.model_api import *

BENEFITS = [
  "snap",
  "wic",
  "ssi",
]

class spm_unit_federal_tax(Variable):
    value_type = float
    entity = SPMUnit
    label = "Federal income tax"
    definition_period = YEAR
    unit = USD

    def formula(spm_unit, period, parameters):
        total_income = add(spm_unit, period, ["taxable_income"])
        return 0.4 * total_income

class flat_tax_reform(Reform):
  def apply(self):
     self.update_variable(spm_unit_federal_tax)
     for removed in BENEFITS: 
        self.neutralize_variable(removed)

# By default, OpenFisca-US uses reported net income in order to exactly match poverty rates.
# This reform defined on the next three lines disables that behaviour, by removing the reported 
# net income values. We apply this 'reform' to both the baseline and reformed simulations.

class turn_on_benefits_simulation(Reform):
  def apply(self):
    self.neutralize_variable("spm_unit_net_income_reported")

baseline = Microsimulation(turn_on_benefits_simulation)
reformed = Microsimulation((turn_on_benefits_simulation, flat_tax_reform))

# revenue 2096.010505407311

/usr/local/lib/python3.7/dist-packages/openfisca_core/holders/holder.py:169: Warning: You cannot set a value for the variable spm_unit_net_income_reported, as it has been neutralized. The value you provided ([28327. 76811. 15350. ... 17282. 78821. 53152.]) will be ignored.
  Warning


In [ ]:
reformed.calc("spm_unit_net_income").sum() / 1e9 - baseline.calc("spm_unit_net_income").sum() / 1e9

-2375.0273671257964